<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo">

DataFrames on a Cluster
=======================

<img src="https://pandas.pydata.org/static/img/pandas.svg"
     align="left"
     width="30%"
     alt="Pandas logo">



This notebook needs the [gcsfs library](https://gcsfs.readthedocs.io).

    pip install gcsfs

## Read single dataframe from the cloud with Pandas

In [1]:
from gcsfs import GCSFileSystem

gcs = GCSFileSystem(token='anon')

gcs.ls('anaconda-public-data/nyc-taxi/csv/')

['anaconda-public-data/nyc-taxi/csv/',
 'anaconda-public-data/nyc-taxi/csv/2014',
 'anaconda-public-data/nyc-taxi/csv/2015',
 'anaconda-public-data/nyc-taxi/csv/2016']

In [1]:

nyc_datatype = {'VendorID': 'string',
                'passenger_count': 'int32',
                'trip_distance': 'float32',
                'pickup_longitude': 'float32',
                'pickup_latitude': 'float32',
                'RateCodeID': 'string',
                'store_and_fwd_flag': 'string',
                'dropoff_longitude': 'float32',
                'dropoff_latitude': 'float32',
                'payment_type': 'string',
                'fare_amount': 'float32',
                'extra': 'float32',
                'mta_tax': 'float32',
                'tip_amount': 'float32',
                'tolls_amount': 'float32',
                'improvement_surcharge': 'float32',
                'total_amount':'float32' }

## Parallelize Pandas with Dask.dataframe


In [2]:
import dask
import cudf
import dask_cudf

import dask.dataframe as dd
#import dask_cudf.DataFrame as ddf
#ddf = dask_cudf.from_cudf(df, npartitions=2)

from dask_cuda import LocalCUDACluster

from dask.distributed import Client, progress, wait


In [3]:
print("Running Dask local cluster")
cluster = LocalCUDACluster()

client = Client(cluster)
client

Running Dask local cluster


2022-12-25 22:46:11,794 - distributed.diskutils - INFO - Found stale lock file and directory '/rapids/notebooks/host/docker-pyspark/dask-docker/workspace/dask-worker-space/worker-xv4_28w7', purging
2022-12-25 22:46:11,794 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 31.36 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42753,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 31.36 GiB
Comm: tcp://127.0.0.1:44015,Total threads: 1
Dashboard: http://127.0.0.1:36059/status,Memory: 31.36 GiB
Nanny: tcp://127.0.0.1:35593,


In [4]:
for i in range(1, 6):
    fname = "data/nyc-taxi/parquet/yellow_tripdata_2014-%02d.parquet" % i
    df1   = cudf.read_parquet(fname)
    ddf1  = dask_cudf.from_cudf(df1, npartitions=2)
    if i==1:
        df = ddf1
    else:
        df = dask_cudf.concat( [df, ddf1], axis=0 )

In [30]:
df1 = cudf.read_parquet("data/nyc-taxi/parquet/yellow_tripdata_2014-01.parquet")
df2 = cudf.read_parquet("data/nyc-taxi/parquet/yellow_tripdata_2015-01.parquet")

#df1 = df1.astype(nyc_datatype)
#df2 = df2.astype(nyc_datatype)

ddf1 = dask_cudf.from_cudf(df1, npartitions=2)
ddf2 = dask_cudf.from_cudf(df2, npartitions=2)

df = dask_cudf.concat( [ddf1, ddf2], axis=0 )

In [5]:
#df = dd.read_parquet("data/nyc_taxi/yellow_tripdata_2020-*.parquet")



In [5]:
df = df.persist()
progress(df)

VBox()

In [6]:
%time wait(df)
%time print(df.passenger_count.sum().compute())


CPU times: user 13 s, sys: 5.23 s, total: 18.3 s
Wall time: 27.3 s
121719790
CPU times: user 26.2 ms, sys: 8.88 ms, total: 35.1 ms
Wall time: 52.8 ms



Dask DataFrames
---------------

*  Coordinate many Pandas DataFrames across a cluster
*  Faithfully implement a subset of the Pandas API
*  Use Pandas under the hood (for speed and maturity)

In [7]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
npartitions=10,,,,,,,,,,,,,,,,,,,
,object,datetime64[us],datetime64[us],int32,float32,float32,float32,object,object,float32,float32,object,float32,float32,float32,float32,float32,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
df.dtypes

VendorID                         object
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                   int32
trip_distance                   float32
pickup_longitude                float32
pickup_latitude                 float32
RateCodeID                       object
store_and_fwd_flag               object
dropoff_longitude               float32
dropoff_latitude                float32
payment_type                     object
fare_amount                     float32
extra                           float32
mta_tax                         float32
tip_amount                      float32
tolls_amount                    float32
improvement_surcharge           float32
total_amount                    float32
dtype: object

In [9]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
__null_dask_index__,,,,,,,,,,,,,,,,,,,
0,CMT,2014-01-09 20:45:25,2014-01-09 20:52:31,1,0.70,-73.994766,40.736828,1,N,-73.982224,40.731789,CRD,6.5,0.0,0.5,1.4,0.0,0.5,8.900000
0,CMT,2014-01-12 11:29:39,2014-01-12 11:34:17,2,0.90,-74.005859,40.725498,1,N,-73.993156,40.727966,CRD,5.5,0.0,0.5,1.0,0.0,0.0,7.000000
0,VTS,2014-01-30 12:25:00,2014-01-30 12:52:00,2,0.87,-74.002251,40.740017,1,<NA>,-73.990646,40.744530,CRD,16.5,0.0,0.5,3.3,0.0,0.0,20.299999
0,VTS,2014-01-31 23:47:00,2014-01-31 23:53:00,2,0.78,-73.989685,40.763580,1,<NA>,-73.986328,40.756119,CRD,5.5,0.0,0.5,1.5,0.0,0.5,8.000000
0,VTS,2014-01-02 17:50:00,2014-01-02 17:58:00,3,1.24,-73.975777,40.757046,1,<NA>,-73.979240,40.746223,CSH,7.0,0.0,0.5,0.0,0.0,1.0,8.500000


In [10]:
%time len(df)

CPU times: user 19.1 ms, sys: 1.26 ms, total: 20.4 ms
Wall time: 28.4 ms


71667210

In [11]:
%time df.passenger_count.sum().compute()

CPU times: user 16 ms, sys: 5.61 ms, total: 21.6 ms
Wall time: 30.1 ms


121719790

In [12]:
# Compute average trip distance grouped by passenger count
df.groupby(df.passenger_count).trip_distance.mean().compute()

passenger_count
7      2.122000
3      2.966308
5      2.989252
0      2.364025
2      3.646793
6      2.912066
4      2.995464
9      4.263714
1      3.000389
8      1.679444
208    0.000000
Name: trip_distance, dtype: float64

### Tip Fraction, grouped by day-of-week and hour-of-day

In [13]:
df2 = df[(df.tip_amount > 0) & (df.fare_amount > 0)]
df2['tip_fraction'] = df2.tip_amount / df2.fare_amount

In [14]:
# Group df.tpep_pickup_datetime by dayofweek and hour
dayofweek = df2.groupby(df2.tpep_pickup_datetime.dt.dayofweek).tip_fraction.mean() 
hour = df2.groupby(df2.tpep_pickup_datetime.dt.hour).tip_fraction.mean()

dayofweek, hour = dask.persist(dayofweek, hour)
progress(dayofweek, hour)

VBox()

### Plot results

This requires matplotlib to be installed

In [15]:
%matplotlib inline

2022-12-25 22:47:13,487 - distributed.worker - WARNING - Compute Failed
Key:       ('assign-d1bb41f7795bc509dfb4ecb21c3e5e9e', 8)
Function:  subgraph_callable-61802fbb-d1ba-4f1b-8840-0a59c8de
args:      (                    VendorID  ... total_amount
__null_dask_index__           ...             
0                        VTS  ...          3.0
0                        CMT  ...         12.0
0                        CMT  ...         10.5
0                        CMT  ...         52.5
0                        VTS  ...          8.5
...                      ...  ...          ...
194395                   CMT  ...         12.5
194395                   CMT  ...         15.0
194395                   CMT  ...         11.5
194395                   CMT  ...         11.4
194395                   CMT  ...         17.5

[7387048 rows x 19 columns])
kwargs:    {}
Exception: "MemoryError('std::bad_alloc: out_of_memory: CUDA error at: /workspace/.conda-bld/work/include/rmm/mr/device/cuda_memory_resource.

In [16]:
pandas_hour = hour.compute().head().to_pandas()
pandas_hour.sort_index(axis=0, inplace=True)
pandas_hour.plot(figsize=(10, 6), title='Tip Fraction by Hour')

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /workspace/.conda-bld/work/include/rmm/mr/device/cuda_memory_resource.hpp:70: cudaErrorMemoryAllocation out of memory